**Corpus de la Lista de espera**

Trabajaran con un conjunto de datos reales correspondiente a interconsultas de la lista de espera NO GES en Chile. Si quieren saber más sobre cómo fueron generados los datos pueden revisar el paper publicado hace unos meses atrás en el workshop de EMNLP, una de las conferencias más importantes de NLP: [https://www.aclweb.org/anthology/2020.clinicalnlp-1.32/](https://www.aclweb.org/anthology/2020.clinicalnlp-1.32/).

Este corpus Chileno está constituido originalmente por 7 tipos de entidades pero por simplicidad en esta competencia trabajarán con las siguientes:

- **Disease**
- **Body_Part**
- **Medication** 
- **Procedures** 
- **Family_Member**

Si quieren obtener más información sobre estas entidades pueden consultar la [guía de anotación](https://plncmm.github.io/annodoc/). Además, mencionar que este corpus está restringido bajo una licencia que permite solamente su uso académico, así que no puede ser compartido más allá de este curso o sin permisos por parte de los autores en caso que quieran utilizarlo fuera. Si este último es el caso entonces pueden escribir directamente al correo: pln@cmm.uchile.cl. Al aceptar los términos y condiciones de la competencia están de acuerdo con los puntos descritos anteriormente.


**Formato ConLL**

Los archivos que serán entregados a ustedes vienen en un formato estándar utilizado en NER, llamado ConLL. No es más que un archivo de texto, que cumple las siguientes propiedades.

- Un salto de linea corresponde a la separación entre oraciones. Esto es importante ya que al entrenar una red neuronal ustedes pasaran una lista de oraciones como input, más conocidos como batches.

- La primera columna del archivo contiene todos los tokens de la partición.

- La segunda columna del archivo contiene el tipo de entidad asociado al token de la primera columna.

- Los tipos de entidades siguen un formato clásico en NER denominado *IOB2*. Si un tipo de entidad comienza con el prefijo "B-" (Beginning) significa que es el token de inicio de una entidad, si comienza con "I-" (Inside) es un token distinto al de inicio y si un token está asociado a la categoría O (Outside) significa que no pertenece a ninguna entidad.

Aquí va un ejemplo:

```
PACIENTE O
PRESENTA O
FRACTURA B-Disease
CORONARIA I-Disease
COMPLICADA I-Disease
EN O
PIE B-Body_Part
IZQUIERDO I-Body_Part
. O
SE O
REALIZA O
INSTRUMENTACION B-Procedure
INTRACONDUCTO I-Procedure
. O
```

Según nuestra definición tenemos las siguientes tres entidades (enumerando desde 0): 

- $(2, 4, Disease)$
- $(6, 7, Body Part)$
- $(11, 12, Procedure)$


Antes de pasar a explicar las reglas, se recomienda visitar los siguientes links para entender bien el baseline de la competencia:

-  [Tagging, and Hidden Markov Models ](http://www.cs.columbia.edu/~mcollins/cs4705-spring2019/slides/tagging.pdf) (slides by Michael Collins), [notes](http://www.cs.columbia.edu/~mcollins/hmms-spring2013.pdf), [video 1](https://youtu.be/-ngfOZz8yK0), [video 2](https://youtu.be/PLoLKQwkONw), [video 3](https://youtu.be/aaa5Qoi8Vco), [video 4](https://youtu.be/4pKWIDkF_6Y)       
-  [Recurrent Neural Networks](slides/NLP-RNN.pdf) | [video 1](https://youtu.be/BmhjUkzz3nk), [video 2](https://youtu.be/z43YFR1iIvk), [video 3](https://youtu.be/7L5JxQdwNJk)


Recuerden que todo el material se encuentra disponible en el [github del curso](https://github.com/dccuchile/CC6205).

# **Introducción**


En el presente trabajo muestra la clasificación de la palabras a traves de Named Entity Recognition (NER). NER consiste en intentar extraer entindades en el traxto a traves de etiquetas pre definidas, como pueden ser nombres de persona, organizaciones, fechas, lugares, entre otros.

  Específicamente, en este trabajo se trata de realizar el reconocimiento de entidades en un dataset clínico que corresponde a la lista de espera no GES en Chile. La idea es clasificar las palabras en alguna de las siguientes 6 etiquetas: `Disease`, para las enfermedades; `Body_part`, para alguna parte del cuerpo, incluye órganos; `Medication`, para cualquier tipo de medicamento que sea nombrado; `Procedures`, hace referencia a todos los tipos de procedimientos que puede someterse un paciente; `Family_Member`, para familiares del paciente y `O`, para cualquier otra palabra que no quepa dentro de alguna de las categorías anteriores.

  Para su realización, empleamos 3 modelos distintos en los que se utilizan distintas arquitecturas de redes neuronales, como lo son las LSTM y GRU. Además se utilizó una biblioteca de embeddings pre entrenados como tipo Glove y Fast Text, para aumetar el accuracy del sistema.

  A continuación se encuentra n las descripciones de los modelos, las métricas de evaluación utilizadas. El diseño experimental, que explica el funcionamiento de los modelos, luego viene la implementación de estos y sus resultados.

###  **Carga de datos y Preprocesamiento**

Para cargar los datos y preprocesarlos usaremos la librería [`torchtext`](https://github.com/pytorch/text). Tener cuidado ya que hace algunos meses esta librería tuvo cambios radicales, quedando las funcionalidades pasadas en un nuevo paquete llamado legacy. Esto ya que si quieren usar más funciones de la librería entonces vean los cambios en la documentación.

En particular usaremos su módulo `data`, el cual según su documentación original provee: 

    - Ability to describe declaratively how to load a custom NLP dataset that's in a "normal" format
    - Ability to define a preprocessing pipeline
    - Batching, padding, and numericalizing (including building a vocabulary object)
    - Wrapper for dataset splits (train, validation, test)


El proceso será el siguiente: 

1. Descargar los datos desde github y examinarlos.
2. Definir los campos (`fields`) que cargaremos desde los archivos.
3. Cargar los datasets.
4. Crear el vocabulario.

In [68]:
# Instalamos torchtext que nos facilitará la vida en el pre-procesamiento del formato ConLL.
# !pip install -U torchtext==0.10.0

# Librerias

In [69]:
import random
import os, shutil
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import gzip
import os
import shutil
import requests

from operator import attrgetter
from torchtext import vocab, datasets ,data
#from torchtext.legacy import data #, datasets
from seqeval.metrics import f1_score, precision_score, recall_score

In [70]:
# Garantizar reproducibilidad de los experimentos
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

#### **Obtener datos**

Descargamos los datos de entrenamiento, validación y prueba en nuestro directorio de trabajo

In [71]:
#%%capture

# !wget https://github.com/dccuchile/CC6205/releases/download/v1.0/train.txt -nc # Dataset de Entrenamiento
# !wget https://github.com/dccuchile/CC6205/releases/download/v1.0/dev.txt -nc    # Dataset de Validación (Para probar y ajustar el modelo)
# !wget https://github.com/dccuchile/CC6205/releases/download/v1.0/test.txt -nc  # Dataset de la Competencia. Estos datos solo contienen los tokens. ¡¡SON LOS QUE DEBEN SER PREDICHOS!!

####  **Fields**

Un `field`:

* Define un tipo de datos junto con instrucciones para convertir el texto a Tensor.
* Contiene un objeto `Vocab` que contiene el vocabulario (palabras posibles que puede tomar ese campo).
* Contiene otros parámetros relacionados con la forma en que se debe numericalizar un tipo de datos, como un método de tokenización y el tipo de Tensor que se debe producir.


Analizemos el siguiente cuadro el cual contiene un ejemplo cualquiera de entrenamiento:


```
El O
paciente O
padece O
de O
cancer B-Disease
de I-Disease
colon I-Disease
. O
```

Cada linea contiene un token y el tipo de entidad asociado en el formato IOB2 ya explicado. Para que `torchtext` pueda cargar estos datos, debemos definir como va a leer y separar los componentes de cada una de las lineas.
Para esto, definiremos un field para cada uno de esos componentes: Las palabras (`TEXT`) y las etiquetas o categorías (`NER_TAGS`).


In [72]:
# Primer Field: TEXT. Representan los tokens de la secuencia
TEXT = data.Field(lower=False) 

# Segundo Field: NER_TAGS. Representan los Tags asociados a cada palabra.
NER_TAGS = data.Field(unk_token=None)
fields = (("text", TEXT), ("nertags", NER_TAGS))

print(fields)

(('text', <torchtext.data.field.Field object at 0x7f98405708e0>), ('nertags', <torchtext.data.field.Field object at 0x7f98405708b0>))


####  **SequenceTaggingDataset**

`SequenceTaggingDataset` es una clase de torchtext diseñada para contener datasets de sequence labeling. Los ejemplos que se guarden en una instancia de estos serán arreglos de palabras asociados con sus respectivos tags.

Por ejemplo, para Part-of-speech tagging:

[I, love, PyTorch, .] estará asociado con [PRON, VERB, PROPN, PUNCT]


La idea es que usando los fields que definimos antes, le indiquemos a la clase cómo cargar los datasets de prueba, validación y test.

In [73]:
train_data, valid_data, test_data = datasets.SequenceTaggingDataset.splits(
    path="./",
    train="corpus_recetas_train.txt",
    validation="corpus_recetas_val.txt",
    test="corpus_recetas_test.txt",
    fields=fields,
    encoding="utf-8",
    separator=" "
)

In [74]:
# valid_data = train_data[int(len(train_data)*0.7):int(len(train_data)*0.8)]
# test_data = train_data[int(len(train_data)*0.8):len(train_data)]
# train_data = train_data[0:int(len(train_data)*0.7)]

In [75]:
print(f"Numero de ejemplos de entrenamiento: {len(train_data)}")
print(f"Número de ejemplos de validación: {len(valid_data)}")
print(f"Número de ejemplos de test (competencia): {len(test_data)}")

Numero de ejemplos de entrenamiento: 500
Número de ejemplos de validación: 250
Número de ejemplos de test (competencia): 250


Visualizemos un ejemplo

In [76]:
random_item_idx = random.randint(0, len(train_data))
random_example = train_data.examples[random_item_idx]
list(zip(random_example.text, random_example.nertags))

[('TRAMADOL', 'B-ACTVPRNCP'),
 ('100', 'O'),
 ('MG/ML', 'O'),
 ('SOLUCIÓN', 'B-ADMIN'),
 ('ORAL', 'I-ADMIN'),
 ('FRASCO', 'O'),
 ('10', 'O'),
 ('ML', 'O'),
 ('5', 'O'),
 ('GOTAS', 'B-ADMIN'),
 ('ORAL', 'I-ADMIN'),
 ('cada', 'B-PERIODICITY'),
 ('8', 'I-PERIODICITY'),
 ('horas', 'I-PERIODICITY'),
 ('durante', 'B-DURATION'),
 ('7', 'I-DURATION'),
 ('dias', 'I-DURATION')]

#### **Construir los vocabularios para el texto y las etiquetas**

Los vocabularios son los objetos que contienen todos los tokens (de entrenamiento) posibles para ambos fields. El siguiente paso consiste en construirlos. Para esto, hacemos uso del método `Field.build_vocab` sobre cada uno de nuestros `fields`. 

In [77]:
TEXT.build_vocab(train_data)
NER_TAGS.build_vocab(train_data)

In [78]:
print(f"Tokens únicos en TEXT: {len(TEXT.vocab)}")
print(f"Tokens únicos en NER_TAGS: {len(NER_TAGS.vocab)}")

Tokens únicos en TEXT: 683
Tokens únicos en NER_TAGS: 10


In [79]:
#Veamos las posibles etiquetas que hemos cargado:
NER_TAGS.vocab.itos

['<pad>',
 'O',
 'B-ADMIN',
 'B-ACTVPRNCP',
 'I-ADMIN',
 'I-PERIODICITY',
 'I-DURATION',
 'B-PERIODICITY',
 'I-ACTVPRNCP',
 'B-DURATION']

Observen que ademas de los tags NER, tenemos \<pad\>, el cual es generado por el dataloader para cumplir con el padding de cada oración.

Veamos ahora los tokens mas frecuentes y especiales:

In [80]:
# Tokens mas frecuentes (Será necesario usar stopwords, eliminar símbolos o nos entregan información (?) )
TEXT.vocab.freqs.most_common(10)

[('ML', 352),
 ('MG', 332),
 ('ORAL', 284),
 ('cada', 279),
 ('COMPRIMIDO', 273),
 ('horas', 267),
 ('1', 235),
 ('SOLUCIÓN', 213),
 ('INYECTABLE', 210),
 ('AMPOLLA', 197)]

In [81]:
# Seteamos algunas variables que nos serán de utilidad mas adelante...
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

PAD_TAG_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]
O_TAG_IDX = NER_TAGS.vocab.stoi['O']

#### **Frecuencia de los Tags**

Visualizemos rápidamente las cantidades y frecuencias de cada tag:

In [82]:
def tag_percentage(tag_counts):
    
    total_count = sum([count for tag, count in tag_counts])
    tag_counts_percentages = [(tag, count, count/total_count) for tag, count in tag_counts]
  
    return tag_counts_percentages

print("Tag Ocurrencia Porcentaje\n")

for tag, count, percent in tag_percentage(NER_TAGS.vocab.freqs.most_common()):
    print(f"{tag}\t{count}\t{percent*100:4.1f}%")

Tag Ocurrencia Porcentaje

O	4122	54.4%
B-ADMIN	789	10.4%
B-ACTVPRNCP	622	 8.2%
I-ADMIN	602	 7.9%
I-PERIODICITY	554	 7.3%
I-DURATION	292	 3.9%
B-PERIODICITY	277	 3.7%
I-ACTVPRNCP	177	 2.3%
B-DURATION	146	 1.9%


#### **Configuramos pytorch y dividimos los datos.**

Importante: si tienes problemas con la ram de la gpu, disminuye el tamaño de los batches

In [83]:
BATCH_SIZE = 32 #16  # disminuir si hay problemas de ram.

# Usar cuda si es que está disponible.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using', device)

# Dividir datos entre entrenamiento y test. Si van a hacer algún sort no puede ser sobre
# el conjunto de testing ya que al hacer sus predicciones sobre el conjunto de test sin etiquetas
# debe conservar el orden original para ser comparado con los golden_labels. 

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device,
    sort=False,
)

Using cuda


#### **Métricas de evaluación**

Además, definiremos las métricas que serán usadas tanto para la competencia como para evaluar el modelo: `precision`, `recall` y `micro f1-score`.
**Importante**: Noten que la evaluación solo se hace para las Named Entities (sin contar 'O'), toda esta funcionalidad nos la entrega la librería seqeval, pueden revisar más documentación aquí: https://github.com/chakki-works/seqeval. No utilicen el código entregado por sklearn para calcular las métricas ya que esta lo hace a nivel de token y no a nivel de entidad.

In [84]:
# Definimos las métricas
def calculate_metrics(preds, y_true, pad_idx=PAD_TAG_IDX, o_idx=O_TAG_IDX):
    """
    Calcula precision, recall y f1 de cada batch.
    """

    # Obtener el indice de la clase con probabilidad mayor. (clases)
    y_pred = preds.argmax(dim=1, keepdim=True)

    # filtramos <pad> para calcular los scores.
    mask = [(y_true != pad_idx)]
    y_pred = y_pred[mask]
    y_true = y_true[mask]

    # traemos a la cpu
    y_pred = y_pred.view(-1).to('cpu').numpy()
    y_true = y_true.to('cpu').numpy()
    y_pred = [[NER_TAGS.vocab.itos[v] for v in y_pred]]
    y_true = [[NER_TAGS.vocab.itos[v] for v in y_true]]
    
    # calcular scores
    f1 = f1_score(y_true, y_pred, mode='strict')
    precision = precision_score(y_true, y_pred, mode='strict')
    recall = recall_score(y_true, y_pred, mode='strict')

    return precision, recall, f1

-------------------

### **Modelo Baseline**

Teniendo ya cargado los datos, toca definir nuestro modelo. Este baseline tendrá una capa de embedding, unas cuantas LSTM y una capa de salida y usará dropout en el entrenamiento.

Este constará de los siguientes pasos: 

1. Definir la clase que contendrá la red.
2. Definir los hiperparámetros e inicializar la red. 
3. Definir el número de épocas de entrenamiento
4. Definir la función de loss.



Recomendamos que para experimentar, encapsules los modelos en una sola variable y luego la fijes en model para entrenarla

In [85]:
# Definir la red
class NER_RNN(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim,
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):

        super().__init__()

        # Capa de embedding
        self.embedding = nn.Embedding(input_dim,
                                      embedding_dim,
                                      padding_idx=pad_idx,
                                      )

        # Capa LSTM
        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional, 
                           dropout = dropout if n_layers > 1 else 0)

        # Capa de salida
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim,
                            output_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        #text = [sent len, batch size]

        # Convertir lo enviado a embedding
        embedded = self.dropout(self.embedding(text))
        
        outputs, (hidden, cell) = self.lstm(embedded)
        #embedded = [sent len, batch size, emb dim]

        # Pasar los embeddings por la rnn (LSTM)

        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]

        # Predecir usando la capa de salida.
        predictions = self.fc(self.dropout(outputs))
        #predictions = [sent len, batch size, output dim]

        return predictions

#### **Hiperparámetros de la red**



In [86]:
# tamaño del vocabulario. recuerden que la entrada son vectores bag of word(one-hot).
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300  # dimensión de los embeddings.
HIDDEN_DIM = 256  # dimensión de la capas LSTM
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 3  # número de capas.
DROPOUT = 0.5
BIDIRECTIONAL = False

# Creamos nuestro modelo.
baseline_model = NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

baseline_model_name = 'baseline'  # nombre que tendrá el modelo guardado...

In [87]:
baseline_n_epochs = 10

#### Definimos la función de loss

In [88]:
# Loss: Cross Entropy
TAG_PAD_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]
baseline_criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

--------------------
### Modelo 1

En estas secciones pueden implementar nuevas redes al modificar los hiperparámetros, la cantidad de épocas de entrenamiento, el tamaño de los batches, loss, optimizador, etc... como también definir nuevas arquitecturas de red (mediante la creación de clases nuevas)


Al final de estas, hay 4 variables, las cuales deben setear con los modelos, épocas de entrenamiento, loss y optimizador que deseen probar.


In [89]:
# Caergamos Glove o fast text
FASTTEXT_FILE = "glove300d.vec"
# Se descargan vectores glove o fasttext del github del dcc
# https://github.com/dccuchile/spanish-word-embeddings

if not os.path.exists(FASTTEXT_FILE):
    print(f"Descargando {FASTTEXT_FILE}")
    url = "http://dcc.uchile.cl/~jperez/word-embeddings/glove-sbwc.i25.vec.gz"
    #url = "https://s06.imfd.cl/04/fasttext-sbwc.vec.gz"
    response = requests.get(url, stream=True)
    try:
        with gzip.open(response.raw, "rb") as f_in:
            with open(FASTTEXT_FILE, "wb") as f_out:
                # Funcion para copiar de un file-like object a otro
                shutil.copyfileobj(f_in, f_out)
    except Exception as e:
        os.remove(FASTTEXT_FILE)
        raise e

 #dimensión es de 300 y tiene 855,380 vectores pre-entrenados. 

In [90]:
embeddings = vocab.Vectors(FASTTEXT_FILE)


In [91]:
TEXT.vocab.set_vectors(*attrgetter("stoi", "vectors", "dim")(embeddings))
#se asocian los tokens de text a los vectores del embedding


In [92]:
BATCH_SIZE = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using', device)

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device,
    sort=False,
)

Using cuda


In [93]:
class Modelo1_RNN(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim,
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):

        super().__init__()

        # Capa de embedding
        self.embedding = nn.Embedding.from_pretrained(
            embedding_weights.clone(), freeze = False)
        # Capa LSTM
        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional, 
                           dropout = dropout if n_layers > 1 else 0)

        # Capa de salida
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim,
                            output_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)
        

    def forward(self, text):

        #text = [sent len, batch size]

        # Convertir lo enviado a embedding
        embedded = self.dropout(self.embedding(text))
        
        outputs, (hidden, cell) = self.lstm(embedded)
        #embedded = [sent len, batch size, emb dim]

        # Pasar los embeddings por la rnn (LSTM)

        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]

        # Predecir usando la capa de salida.
        predictions = self.fc(self.dropout(outputs))
        #predictions = [sent len, batch size, output dim]

        return predictions

In [94]:
#Parametros para el modelo 1
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 196
OUTPUT_DIM = len(NER_TAGS.vocab)
embedding_weights = TEXT.vocab.vectors

N_LAYERS = 3
DROPOUT = 0.15
BIDIRECTIONAL = True

TAG_PAD_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]


In [95]:
modelo_1 = Modelo1_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, 
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

model_name_1 = 'Modelo_1'
n_epochs_1 = 100 #no importa porq hay early stop
criterion_1 = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

---------------

### Modelo 2

In [96]:
BATCH_SIZE = 62 # disminuir si hay problemas de ram.

# Usar cuda si es que está disponible.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using', device)

# Dividir datos entre entrenamiento y test. Si van a hacer algún sort no puede ser sobre
# el conjunto de testing ya que al hacer sus predicciones sobre el conjunto de test sin etiquetas
# debe conservar el orden original para ser comparado con los golden_labels. 

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device,
    sort=False,
)

Using cuda


In [97]:
class Modelo2_RNN(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim,
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):

        super().__init__()

        # Capa de embedding
        self.embedding = nn.Embedding.from_pretrained(embedding_weights.clone(), freeze = False)

        # Capa LSTM
        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional, 
                           dropout = dropout if n_layers > 1 else 0)

        # Capa de salida
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim,
                            output_dim)
        #self.relu = nn.ReLU()

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        #text = [sent len, batch size]

        # Convertir lo enviado a embedding
        embedded = self.dropout(self.embedding(text))
        
        outputs, (hidden, cell) = self.lstm(embedded)
        #embedded = [sent len, batch size, emb dim]

        # Pasar los embeddings por la rnn (LSTM)

        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]

        # Predecir usando la capa de salida.
        predictions = self.fc(self.dropout(outputs))
        #predictions = [sent len, batch size, output dim]

        return predictions

In [98]:
# tamaño del vocabulario. recuerden que la entrada son vectores bag of word(one-hot).
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300  # dimensión de los embeddings.
HIDDEN_DIM = 196  # dimensión de la capas LSTM
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 2  # número de capas.
DROPOUT = 0.4
BIDIRECTIONAL = True
TAG_PAD_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]

In [99]:
modelo_2 = Modelo2_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, 
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

model_name_2 = 'Modelo_2'
n_epochs_2 = 30 #no importa porq hay early stop
criterion_2 = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

---------------


### Modelo 3

In [100]:
# Para crear la red debemos heredar desde nn.Module
class GruNet(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim,
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):

      super().__init__()

      # Capa de embedding
      self.embedding = nn.Embedding(input_dim,
                                    embedding_dim,
                                    padding_idx=pad_idx,
                                    )
      
      
      # Capa GRU
      self.gru = nn.GRU(embedding_dim, hidden_dim, n_layers, batch_first=True, dropout = dropout if n_layers > 1 else 0, bidirectional=bidirectional)

      # Capa de salida
      self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)


      #self.relu = nn.ReLU()

      # Dropout
      self.dropout = nn.Dropout(dropout)




    # Definimos las operaciones de las capas sobre el input en el forward.
    def forward(self, text): 

      embedded = self.embedding(text)

      outputs, hidden = self.gru(embedded)

      predictions = self.fc(self.dropout(outputs))


      return predictions


In [101]:
# tamaño del vocabulario. recuerden que la entrada son vectores bag of word(one-hot).
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300  # dimensión de los embeddings.
HIDDEN_DIM = 256  # dimensión de la capas LSTM
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 2  # número de capas.
DROPOUT = 0.3
BIDIRECTIONAL = False

# Creamos nuestro modelo.
modelo_3 = GruNet(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

model_name_3 = 'Gru_Model'  # nombre que tendrá el modelo guardado...

n_epochs_3 = 10


# Loss: Cross Entropy
TAG_PAD_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]
criterion_3 = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)



------
### **Entrenamos y evaluamos**


**Importante** : Fijen el modelo, el número de épocas de entrenamiento, la loss y el optimizador que usarán para entrenar y evaluar en las siguientes variables!!!

In [102]:
model = baseline_model
model_name = baseline_model_name
criterion = baseline_criterion
n_epochs = baseline_n_epochs

In [103]:
# model = modelo_1
# model_name = model_name_1
# criterion = criterion_1
# n_epochs = n_epochs_1

# # Optimizador
# optimizer = optim.Adam(model.parameters())

In [104]:
# model = modelo_2
# model_name = model_name_2
# criterion = criterion_2
# n_epochs = n_epochs_2

# # Optimizador
# #optimizer = optim.Adam(model.parameters(),lr=0.3)
# optimizer = optim.SGD(model.parameters(), lr=0.3, momentum=0.9)

In [105]:

# model = modelo_3
# model_name = model_name_3
# criterion = criterion_3
# n_epochs = n_epochs_3

# # Optimizador
# optimizer = optim.Adam(model.parameters())


In [106]:
a ='123'
a[:2]

'12'

In [107]:
def function(fecha1,fecha2):
    Año_1=int(str(fecha1)[:2])
    Mes_1=int(str(fecha1)[-2:])
    Año_2=int(str(fecha2)[:2])
    Mes_2=int(str(fecha2)[-2:])

    dif=12*(Año_2-Año_1)+(Mes_2-Mes_1)
    return dif 



#### **Inicializamos la red**

Iniciamos los pesos de la red de forma aleatoria (Usando una distribución normal).


In [108]:
def init_weights(m):
    # Inicializamos los pesos como aleatorios
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.1) 
        
    # Seteamos como 0 los embeddings de UNK y PAD.
    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
        
model.apply(init_weights)

NER_RNN(
  (embedding): Embedding(683, 300, padding_idx=1)
  (lstm): LSTM(300, 256, num_layers=3, dropout=0.5)
  (fc): Linear(in_features=256, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [109]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')

El modelo actual tiene 1,831,534 parámetros entrenables.


Notar que definimos los embeddings que representan a \<unk\> y \<pad\>  como [0, 0, ..., 0]

#### **Definimos el optimizador**

In [110]:
# Optimizador
optimizer = optim.Adam(model.parameters())

#### **Enviamos el modelo a cuda**


In [111]:
# Enviamos el modelo y la loss a cuda (en el caso en que esté disponible)
model = model.to(device)
criterion = criterion.to(device)

#### **Definimos el entrenamiento de la red**

Algunos conceptos previos: 

- `epoch` : una pasada de entrenamiento completa de una dataset.
- `batch`: una fracción de la época. Se utilizan para entrenar mas rápidamente la red. (mas eficiente pasar n datos que uno en cada ejecución del backpropagation)

Esta función está encargada de entrenar la red en una época. Para esto, por cada batch de la época actual, predice los tags del texto, calcula su loss y luego hace backpropagation para actualizar los pesos de la red.

Observación: En algunos comentarios aparecerá el tamaño de los tensores entre corchetes

In [112]:
def train(model, iterator, optimizer, criterion):
    

    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.train()

    # Por cada batch del iterador de la época:
    for batch in iterator:

        # Extraemos el texto y los tags del batch que estamos procesado
        text = batch.text
        tags = batch.nertags

        # Reiniciamos los gradientes calculados en la iteración anterior
        optimizer.zero_grad()

        #text = [sent len, batch size]

        # Predecimos los tags del texto del batch.
        predictions = model(text)

        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]

        # Reordenamos los datos para calcular la loss
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)

        #predictions = [sent len * batch size, output dim]



        # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
        loss = criterion(predictions, tags)
        
        # Calculamos el accuracy
        precision, recall, f1 = calculate_metrics(predictions, tags)

        # Calculamos los gradientes
        loss.backward()

        # Actualizamos los parámetros de la red
        optimizer.step()

        # Actualizamos el loss y las métricas
        epoch_loss += loss.item()
        epoch_precision += precision
        epoch_recall += recall
        epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(
        iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)

#### **Definimos la función de evaluación**

Evalua el rendimiento actual de la red usando los datos de validación. 

Por cada batch de estos datos, calcula y reporta el loss y las métricas asociadas al conjunto de validación. 
Ya que las métricas son calculadas por cada batch, estas son retornadas promediadas por el número de batches entregados. (ver linea del return)

In [113]:
def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.eval()

    # Indicamos que ahora no guardaremos los gradientes
    with torch.no_grad():
        # Por cada batch
        for batch in iterator:

            text = batch.text
            tags = batch.nertags

            # Predecimos
            predictions = model(text)

            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)

            # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
            loss = criterion(predictions, tags)

            # Calculamos las métricas
            precision, recall, f1 = calculate_metrics(predictions, tags)

            # Actualizamos el loss y las métricas
            epoch_loss += loss.item()
            epoch_precision += precision
            epoch_recall += recall
            epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(
        iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)

In [114]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


#### **Entrenamiento de la red**

En este cuadro de código ejecutaremos el entrenamiento de la red.
Para esto, primero definiremos el número de épocas y luego por cada época, ejecutaremos `train` y `evaluate`.

**Importante: Reiniciar los pesos del modelo**

Si ejecutas nuevamente esta celda, se seguira entrenando el mismo modelo una y otra vez. 
Para reiniciar el modelo se debe ejecutar nuevamente la celda que contiene la función `init_weights`



In [115]:
#Agregar early stop
best_valid_loss = float('inf')
best_train_loss = float('inf')

arrayTrainLoss = []
arrayValidLoss = []

for epoch in range(n_epochs):

    start_time = time.time()

    # Recuerdo: train_iterator y valid_iterator contienen el dataset dividido en batches.

    # Entrenar
    train_loss, train_precision, train_recall, train_f1 = train(
        model, train_iterator, optimizer, criterion)

    # Evaluar (valid = validación)
    valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(
        model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    #Nuevo **
    arrayTrainLoss.append(train_loss)
    arrayValidLoss.append(valid_loss)

  #Acá nos aseguramos de que entrene al menos 4 épocas antes de detener el entrenamiento.
    if epoch < 5:
      if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        #nuevo
        best_train_loss = train_loss
        torch.save(model.state_dict(), '{}.pt'.format(model_name))
        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(
            f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}'
        )
        print(
            f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} |  Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
        )
      else :
        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(
            f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}'
        )
        print(
            f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} |  Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
        )

  #Early stop, ve que no vaya en aumento, según el avg de las últimas 4 épocas
    else:
      if train_loss < np.mean(arrayTrainLoss[-4:]) and valid_loss > np.mean(arrayValidLoss[-4:]):
        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(
            f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}'
        )
        print(
            f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} |  Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
        )
        print('Early Stop')
        break

      else:
        best_valid_loss = valid_loss
        #nuevo
        best_train_loss = train_loss
        torch.save(model.state_dict(), '{}.pt'.format(model_name))
        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(
            f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}'
        )
        print(
            f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} |  Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
        )

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 1.768 | Train f1: 0.04 | Train precision: 0.13 | Train recall: 0.03
	 Val. Loss: 1.466 |  Val. f1: 0.00 |  Val. precision: 0.17 | Val. recall: 0.00
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 1.436 | Train f1: 0.04 | Train precision: 0.16 | Train recall: 0.02
	 Val. Loss: 1.288 |  Val. f1: 0.04 |  Val. precision: 0.15 | Val. recall: 0.03
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 1.252 | Train f1: 0.16 | Train precision: 0.31 | Train recall: 0.11
	 Val. Loss: 1.132 |  Val. f1: 0.27 |  Val. precision: 0.62 | Val. recall: 0.19
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 1.137 | Train f1: 0.28 | Train precision: 0.48 | Train recall: 0.20
	 Val. Loss: 0.976 |  Val. f1: 0.36 |  Val. precision: 0.62 | Val. recall: 0.26
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 0.971 | Train f1: 0.39 | Train precision: 0.58 | Train recall: 0.30
	 Val. Loss: 0.866 |  Val. f1: 0.46 |  Val. precision: 0.64 | Val. recall: 0.36
Epoch: 06 | Epoch Time: 0m 0s
	Train Loss: 0.

**Importante**: Recuerden que el último modelo entrenado no es el mejor (probablemente esté *overfitteado*), si no el que guardamos con la menor loss del conjunto de validación. Este problema lo pueden solucionar con *early stopping*.
Para cargar el mejor modelo entrenado, ejecuten la siguiente celda.



In [116]:
# cargar el mejor modelo entrenado.
model.load_state_dict(torch.load('{}.pt'.format(model_name)))

<All keys matched successfully>

In [117]:
# Limpiar ram de cuda
torch.cuda.empty_cache()

#### **Evaluamos el set de validación con el modelo final**

Estos son los resultados de predecir el dataset de evaluación con el *mejor* modelo entrenado.

In [118]:
valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(
    model, valid_iterator, criterion)

print(
    f'Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} | Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
)

Val. Loss: 0.307 |  Val. f1: 0.88 | Val. precision: 0.90 | Val. recall: 0.87


### **Predecir datos para la competencia**

Ahora, a partir de los datos de **test** y nuestro modelo entrenado, vamos a predecir las etiquetas que serán evaluadas en la competencia.

In [119]:
def predict_labels(model, iterator, criterion, fields=fields):

    # Extraemos los vocabularios.
    text_field = fields[0][1]
    nertags_field = fields[1][1]
    tags_vocab = nertags_field.vocab.itos
    words_vocab = text_field.vocab.itos

    model.eval()

    predictions = []

    with torch.no_grad():

        for batch in iterator:

            text_batch = batch.text
            text_batch = torch.transpose(text_batch, 0, 1).tolist()

            # Predecir los tags de las sentences del batch
            predictions_batch = model(batch.text)
            predictions_batch = torch.transpose(predictions_batch, 0, 1)

            # por cada oración predicha:
            for sentence, sentence_prediction in zip(text_batch,
                                                     predictions_batch):
                for word_idx, word_predictions in zip(sentence,
                                                      sentence_prediction):
                    # Obtener el indice del tag con la probabilidad mas alta.
                    argmax_index = word_predictions.topk(1)[1]

                    current_tag = tags_vocab[argmax_index]
                    # Obtenemos la palabra
                    current_word = words_vocab[word_idx]

                    if current_word != '<pad>':
                        predictions.append([current_word, current_tag])
                predictions.append(['EOS', 'EOS'])


    return predictions


predictions = predict_labels(model, test_iterator, criterion)

In [120]:
predictions

[['FENITOÍNA', 'B-ACTVPRNCP'],
 ['250', 'O'],
 ['MG/5', 'O'],
 ['ML', 'O'],
 ['SOLUCIÓN', 'B-ADMIN'],
 ['INYECTABLE', 'I-ADMIN'],
 ['FRASCO', 'O'],
 ['AMPOLLA', 'O'],
 ['5', 'O'],
 ['ML', 'O'],
 ['2', 'O'],
 ['UNIDAD', 'O'],
 ['INTRAMUSCULAR', 'O'],
 ['cada', 'B-PERIODICITY'],
 ['8', 'I-PERIODICITY'],
 ['horas', 'I-PERIODICITY'],
 ['EOS', 'EOS'],
 ['METRONIDAZOL', 'B-ACTVPRNCP'],
 ['500', 'O'],
 ['MG/100', 'O'],
 ['ML', 'O'],
 ['SOLUCIÓN', 'B-ADMIN'],
 ['INYECTABLE', 'I-ADMIN'],
 ['UNIDAD', 'O'],
 ['100', 'O'],
 ['ML', 'O'],
 ['1', 'O'],
 ['FRASCO', 'O'],
 ['PARENTERAL', 'O'],
 ['cada', 'B-PERIODICITY'],
 ['8', 'I-PERIODICITY'],
 ['horas', 'I-PERIODICITY'],
 ['durante', 'B-DURATION'],
 ['3', 'I-DURATION'],
 ['dias', 'I-DURATION'],
 ['EOS', 'EOS'],
 ['DEXAMETASONA', 'B-ACTVPRNCP'],
 ['4', 'O'],
 ['MG/ML', 'O'],
 ['SOLUCIÓN', 'B-ADMIN'],
 ['INYECTABLE', 'I-ADMIN'],
 ['AMPOLLA', 'O'],
 ['1', 'O'],
 ['ML', 'O'],
 ['2', 'O'],
 ['UNIDAD', 'O'],
 ['<unk>', 'O'],
 ['cada', 'B-PERIODICITY'],
 [

### **Generar el archivo para la submission**

No hay problema si aparecen unk en la salida. Estos no son relevantes para evaluarlos, usamos solo los tags.

In [121]:
if (os.path.isfile('./predictions.zip')):
    os.remove('./predictions.zip')

if (not os.path.isdir('./predictions')):
    os.mkdir('./predictions')

else:
    # Eliminar predicciones anteriores:
    shutil.rmtree('./predictions')
    os.mkdir('./predictions')

f = open('predictions/predictions.txt', 'w')
for i, (word, tag) in enumerate(predictions[:-1]):
    if word=='EOS' and tag=='EOS': f.write('\n')
    else: 
      if i == len(predictions[:-1])-1:
        f.write(word + ' ' + tag)
      else: f.write(word + ' ' + tag + '\n')

f.close()

a = shutil.make_archive('predictions', 'zip', './predictions')